In [1]:
from gui.Main import Main
main=Main()
import logging

import sqlalchemy_dao
from sqlalchemy_dao import Dao

from conf.ConfigReader import ConfigReader
from db.ORMs import Filter
from gui.Workflow import Workflow
from utils.AnnotationTypeDef import AnnotationTypeDef
from utils.IntroStep import IntroStep
from utils.KeywordsFiltering import KeywordsFiltering
from utils.KeywordsUMLSExtender import KeywordsUMLSExtender
from utils.KeywordsUMLSExtenderSetup import KeywordsUMLSExtenderSetup
from gui.PreviousNextWidgets import PreviousNextHTML
from utils.ReviewRBInit import ReviewRBInit
from utils.ReviewRBLoop import ReviewRBLoop
from utils.ReviewMLInit import ReviewMLInit
from utils.ReviewMLLoop import ReviewMLLoop
from models.logistic.LogisticBOWClassifier import LogisticBOWClassifier
from utils.DataSetChooser import DataSetChooser

logging.getLogger().setLevel(logging.DEBUG)

ConfigReader('../conf/smartanno_conf.json')

wf = Workflow(config_file=ConfigReader.config_file)
wf.api_key = ConfigReader.getValue("api_key")
wf.dao = Dao('sqlite+pysqlite:///../data/test.sqlite', sqlalchemy_dao.POOL_DISABLED)
wf.task_name = 'language'
wf.append(AnnotationTypeDef(
    '<h3>Annotation types:</h3><p>List all the types you want to identify below. Each type per line.<br/>If you'
    'have too many types, try set up them separately, so that you won&apos;t need to choose from a long list '
    'for each sample. </p>', name='types'))
wf.append(KeywordsFiltering(
    name='keywords'))
wf.append(DataSetChooser(name='dataset_chooser',description='<h4>Choose which dateaset you want to use: </h4>'))
rb=ReviewRBInit(name="rb_review_init")
wf.append(rb)
rv = ReviewRBLoop(name='rb_review', rush_rule='../conf/rush_rules.tsv')
wf.append(rv)
wf.append(PreviousNextHTML('<h2>Congratuations!</h2><h4>You have finished the initial review on the rule-base preannotations. </h4>',
                       name='intro'))
wf.append(ReviewMLInit(name='ml_review_init'))
wf.append(ReviewMLLoop(name='ml_review',ml_classifier_cls=LogisticBOWClassifier))

wf.start()
wf.steps[0].complete()
wf.steps[1].complete()
wf.steps[2].toggle.value='n2c2_sents'
wf.steps[2].complete()

VBox(children=(IntProgress(value=5, description='Progress: 5/100', layout=Layout(height='14px', width='90%'), min=1, style=ProgressStyle(description_width='initial')), HTML(value='<table style="width:100%"><tr><th>DOC_ID</th><th>DATE</th><th>REF_DATE</th></tr><tr><td>4399</td><td>2141-08-31</td><td>2141-08-31</td></tr></table><div id="d1" style="overflow-y: scroll; height:200px;border:1px solid;border-color:#e5e8e8; ">Will <span class="highlighter" style="background-color:  None ">discuss</span> risks/benefits of coumadin w/ patient and PCP.<br/><br/><br/><br/>#) Troponin leak: In the absence of chest pain/angina or concerning EKG changes, this likely represents heart strain in the context of chronic kidney disease.</div>'), ClickResponsiveToggleButtons(button_style='success', index=3, layout=Layout(width='70%'), options=('Eng', 'PossibleEng', 'NotEng', 'Irrelevant'), value='Irrelevant'), Label(value='', layout=Layout(height='10px')), IntProgress(value=1, layout=Layout(height='10px', left='1%', width='98%'), max=1, style=ProgressStyle(bar_color='GAINSBORO')), Label(value='', layout=Layout(height='1px')), HBox(children=(Button(description='Previous', layout=Layout(left='100px', width='90px'), style=ButtonStyle()), Button(description='Next', layout=Layout(left='100px', width='90px'), style=ButtonStyle()), Button(description='Complete', layout=Layout(left='100px', width='90px'), style=ButtonStyle())), layout=Layout(left='10%', width='80%'))))

In [2]:
wf.steps[2].complete()

VBox(children=(HTML(value='<h3>Start sampling for reviewing</h3>'), HTML(value='<h4>Previously Sampled Data:</h4><p>We found <b>200</b> previously sampled documents of this task in the database.</p>'), ToggleButtons(description='What to do with previously sampled data? ', index=1, options=('Remove them', 'Keep them'), style=ToggleButtonsStyle(description_width='initial'), value='Keep them'), HTML(value='<p>If you choose <span style="background-color:  #E8E8E8">"Keep them"</span>, then sample size you set below will be the amount of extra documents to be added.'), Label(value='', layout=Layout(height='10px')), IntProgress(value=1, layout=Layout(height='10px', left='1%', width='98%'), max=1, style=ProgressStyle(bar_color='GAINSBORO')), Label(value='', layout=Layout(height='1px')), HTML(value='<h4>Sample size:</h4>'), BoundedIntText(value=0, description='Total documents you want to sample:', max=122717, style=DescriptionStyle(description_width='initial')), Label(value='', layout=Layout(height='10px')), IntProgress(value=1, layout=Layout(height='10px', left='1%', width='98%'), max=1, style=ProgressStyle(bar_color='GAINSBORO')), Label(value='', layout=Layout(height='1px')), HTML(value='<h4>Percentage to Filter: </h4><p>Choose how many percent of the documents you want to contain the filter keywords. The rest percentage will be sampled randomly from the documents that do not have any filter keywords.</p>'), IntSlider(value=70, continuous_update=False, step=5), Label(value='', layout=Layout(height='10px')), IntProgress(value=1, layout=Layout(height='10px', left='1%', width='98%'), max=1, style=ProgressStyle(bar_color='GAINSBORO')), Label(value='', layout=Layout(height='1px')), HTML(value="<h4>Summary:</h4><table  class='table table-striped table-bordered table-hover '><thead><tr><th>Possible document types based on keywords</th><th>Total # of documents</th><th># of documents haven&apos;t been sampled</th><th># of documents will be sampled</th></tr></thead><tbody><tr><td>Eng</td><td>22</td><td>1</td><td><b>0</b></td> </tr><tr><td>PossibleEng</td><td>172</td><td>136</td><td><b>0</b></td> </tr><tr><td>NotEng</td><td>36</td><td>18</td><td><b>0</b></td> </tr><tr><td>Irrelevant</td><td>902</td><td>877</td><td><b>0</b></td> </tr><tr><td>not_contain</td><td>121785</td><td>121685</td><td><b>0</b></td> </tr><tr><td><b>Total</b></td><td>122917</td><td>122717</td><td><b>0</b></td> </tr></tbody></table><sup>Note: This summary is simply based on keywords search (does not take prevously reviewed true types into consideration). <br/>Each document is counted once. If one document has keywords in multiple types, this document will be counted only in the top type.</sup>"), Label(value='', layout=Layout(height='10px')), IntProgress(value=1, layout=Layout(height='10px', left='1%', width='98%'), max=1, style=ProgressStyle(bar_color='GAINSBORO')), Label(value='', layout=Layout(height='1px')), HBox(children=(Button(description='Previous', layout=Layout(width='90px'), style=ButtonStyle(button_color='SILVER')), Button(description='Next', layout=Layout(left='100px', width='90px'), style=ButtonStyle(button_color='SANDYBROWN'))), layout=Layout(left='10%', width='80%'))))

In [3]:
wf.steps[3].un_reviewed

115

In [4]:
sampler=wf.steps[3].sampler

In [5]:
list=[len(sampler.all_contain_ids), 
      len(sampler.previous_sampled_ids),
     sampler.available_not_contain,
     ]
list

[1132, 200, 121785]

In [6]:
{type_name:len(ids) for type_name,ids in sampler.new_ids.items()}

{'Eng': 1, 'Irrelevant': 877, 'NotEng': 18, 'PossibleEng': 136}

In [7]:
{type_name:len(ids)-len(sampler.new_ids[type_name]) for type_name,ids in sampler.grouped_ids.items()}

{'Eng': 21, 'Irrelevant': 25, 'NotEng': 18, 'PossibleEng': 36}

In [8]:
len(sampler.all_contain_ids.intersection(sampler.previous_sampled_ids))

100